<h1>Qualità delle scuole</h1>

Lo scopo principale del programma è quello di estrarre un 'rank', ovvero una valutazione della scuola, che si basa su riscaldamento, presenza di macro ambienti funzionali e presenza di barriere architettoniche.

I dataset relativi alla qualità delle scuole appartengono all'anno 17/18, pertanto possono esserci delle differenze, che abbiamo corretto negli altri file.


Librerie utilizzate:

In [10]:
import csv
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
from contextlib import closing
import requests 
import time
import sys


Successivamente andiamo a prelevare i 3 dataset e inserirli in dei DataFrame, che verranno ordinati per codice scuola.
Tramite loc, andiamo a prelevare solo le righe il cui codice scuola fa riferimento ad una scuola di Palermo:

<i>Il PA presente nel contains indica Palermo, pertanto si possono utilizzare anche altre sigle, come CT per Catania e così via.</i>

In [11]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
# la sigla identifica direttamente il comune, eliminando le provincie
# es: PA indica tutte le scuole appartenenti a Palermo come citta
sigla = 'PA'

ambiente = pd.read_csv(
    'http://dati.istruzione.it/opendata/opendata/catalogo/elements1/leaf/EDIAMBFUNZSTA20171820180925.csv')
barriera = pd.read_csv(
    'http://dati.istruzione.it/opendata/EDISUPBARARCSTA20171820180925.csv')
riscaldamento = pd.read_csv(
    'http://dati.istruzione.it/opendata/EDITIPORISCSTA20171820180925.csv')

ambiente = ambiente.sort_values(by=['CODICESCUOLA'])
riscaldamento = riscaldamento.sort_values(by=['CODICESCUOLA'])
barriera = barriera.sort_values(by=['CODICESCUOLA'])


Costruiamo i vari DataFrame:

In [12]:
#Ambienti
df1['CODICESCUOLA'] = ambiente['CODICESCUOLA']
df1['CODICEEDIFICIO'] = ambiente['CODICEEDIFICIO']
df1['SPAZIDIDATTICI']=ambiente['SPAZIDIDATTICI']
df1['AULAMAGNA']=ambiente['AULAMAGNA']
df1['MENSA']=ambiente['MENSA']
df1['PALESTRAPISCINA']=ambiente['PALESTRAPISCINA']
df1['SPAZIAMMINISTRATIVI']=ambiente['SPAZIAMMINISTRATIVI']

#Barriere
df2['CODICESCUOLA'] = barriera['CODICESCUOLA']
df2['CODICEEDIFICIO'] = barriera['CODICEEDIFICIO']
df2['SUPERAMENTOBARRIEREARCH']=barriera['SUPERAMENTOBARRIEREARCH']
df2['ACCESSORAMPE']=barriera['ACCESSORAMPE']
df2['SCALENORMA']=barriera['SCALENORMA']
df2['ASCENSOREDISABILI']=barriera['ASCENSOREDISABILI']
df2['PIATTAFORMAELEVATRICE']=barriera['PIATTAFORMAELEVATRICE']
df2['SERVIZIIGIENICIDISABILI']=barriera['SERVIZIIGIENICIDISABILI']
df2['PORTELARGHEZZADISABILI']=barriera['PORTELARGHEZZADISABILI']
df2['PERCORSIINTERNIDISABILI']=barriera['PERCORSIINTERNIDISABILI']
df2['PERCORSIESTERNIDISABILI']=barriera['PERCORSIESTERNIDISABILI']
df2['ALTRIACCORGIMENTIDISABILI']=barriera['ALTRIACCORGIMENTIDISABILI']

#Riscaldamento
df3['CODICESCUOLA'] = riscaldamento['CODICESCUOLA']
df3['CODICEEDIFICIO'] = riscaldamento['CODICEEDIFICIO']
df3['IMPIANTORISCALDAMENTO']=riscaldamento['IMPIANTORISCALDAMENTO']
df3['CENTRALIZZATOOLIOCOMBUSTIBILI']=riscaldamento['CENTRALIZZATOOLIOCOMBUSTIBILI']
df3['CENTRALIZZATOGASOLIO']=riscaldamento['CENTRALIZZATOGASOLIO']
df3['CENTRALIZZATOMETANO']=riscaldamento['CENTRALIZZATOMETANO']
df3['CENTRALIZZATOGPL']=riscaldamento['CENTRALIZZATOGPL']
df3['CENTRALIZZATOARIA']=riscaldamento['CENTRALIZZATOARIA']
df3['CORPISCALDANTIELETTRICIAUTONOMI']=riscaldamento['CORPISCALDANTIELETTRICIAUTONOMI']
df3['TELERISCALDAMENTO']=riscaldamento['TELERISCALDAMENTO']
df3['CONDIZIONAMENTOVENTILAZIONE']=riscaldamento['CONDIZIONAMENTOVENTILAZIONE']
df3['RISCALDAMENTOALTRANATURA']=riscaldamento['RISCALDAMENTOALTRANATURA']

I dataset presentano alcune disomogeneità, corrette con una serie di replace, che verranno utilizzate anche per l'ontologia:

<i>In sostanza, abbiamo identificato la presenza o l'assenza di una determinata infrastruttura come True e False, in modo da trattarla come valore boolean.</i>

In [13]:
#Ambienti
df1=df1.replace({'-':'False'}, regex=True)
df1=df1.replace({'Non Comunicato':'False'}, regex=True)
df1=df1.replace({'Non Esiste':'False'}, regex=True)
df1=df1.replace({'Esiste':'True'}, regex=True)
df1=df1.replace({'SI':'True'}, regex=True)
df1=df1.replace({'NO':'False'}, regex=True)

#Barriere
df2=df2.replace({'SI':'True'}, regex=True)
df2=df2.replace({'NO':'False'}, regex=True)
df2=df2.replace({'-':'False'}, regex=True)

#Riscaldamento
df3=df3.replace({'SI':'True'}, regex=True)
df3=df3.replace({'NO':'False'}, regex=True)
df3=df3.replace({'-':'False'}, regex=True)


Alcuni campi presentano delle stringhe che descrivono il tipo di attrezzatura (diversa dagli altri campi), che abbiamo generalizzato allo stesso modo come True e False:

In [14]:
df2['ALTRIACCORGIMENTIDISABILI'] = df2['ALTRIACCORGIMENTIDISABILI'].replace({' ':''}, regex=True)
df3['RISCALDAMENTOALTRANATURA'] = df3['RISCALDAMENTOALTRANATURA'].replace({' ':''}, regex=True)
df2['ALTRIACCORGIMENTIDISABILI'] = df2['ALTRIACCORGIMENTIDISABILI'].replace({'[^a-zA-Z0-9]':''}, regex=True)
df3['RISCALDAMENTOALTRANATURA'] = df3['RISCALDAMENTOALTRANATURA'].replace({'[^a-zA-Z0-9]':''}, regex=True)
df2['ALTRIACCORGIMENTIDISABILI'] = df2['ALTRIACCORGIMENTIDISABILI'].replace({'\w+':'True'}, regex=True)
df3['RISCALDAMENTOALTRANATURA'] = df3['RISCALDAMENTOALTRANATURA'].replace({'\w+':'True'}, regex=True)

Output intermedi:

In [15]:
df1.to_csv('Output_ambiente.csv', index=False)
df2.to_csv('Output_barriere.csv', index=False)
df3.to_csv('Output_riscaldamento.csv', index=False)

Dopo aver pulito i 3 dataset, ne effettuiamo un merge per codice scuola e codice edificio, per poi ordinarle per codice scuola:

In [16]:
s1 = pd.merge(df1, df2, how='inner', on=['CODICESCUOLA','CODICEEDIFICIO'])
s2 = pd.merge(s1, df3, how='inner', on=['CODICESCUOLA','CODICEEDIFICIO'])
s2=s2.sort_values(by=['CODICESCUOLA'])

#Elimina tutte le scuole che non sono di Palermo
s2 = s2.loc[s2['CODICESCUOLA'].str.contains(r'^'+sigla+r'\w+')]

I dataset identificano la presenza o l'assenza di una determinata infrastruttura come True o False, quindi 1 o 0.
Il rank viene calcolato effettuando una media aritmetica: <br>
<b>Somma di ogni 'SI' presente / numero dei servizi totali </b> <br>
<i>Il \* 10 serve per ottenere un numero compreso tra 1 e 10.</i>

Infine, effettuiamo un round fino alla seconda cifra decimale e salviamo il file.

In [17]:
'''Prende tutti i campi necessari tranne quello del codice scuola e codice edificio, che non vengono 
utilizzati per il calcolo del rank
'''
num_elementi = len(s2.columns)
num_elementi -=2

s2['RANK'] = (s2.iloc[:,2:] == 'True').sum(axis=1)/num_elementi*10

s2['RANK'] = s2['RANK'].apply(lambda x: round(x, 2))


s2.to_csv('Output_rank.csv', index=False)

In [18]:
s2

,CODICESCUOLA,CODICEEDIFICIO,SPAZIDIDATTICI,AULAMAGNA,MENSA,PALESTRAPISCINA,SPAZIAMMINISTRATIVI,SUPERAMENTOBARRIEREARCH,ACCESSORAMPE,SCALENORMA,...,CENTRALIZZATOOLIOCOMBUSTIBILI,CENTRALIZZATOGASOLIO,CENTRALIZZATOMETANO,CENTRALIZZATOGPL,CENTRALIZZATOARIA,CORPISCALDANTIELETTRICIAUTONOMI,TELERISCALDAMENTO,CONDIZIONAMENTOVENTILAZIONE,RISCALDAMENTOALTRANATURA,RANK
34032,PAAA003039,820534065,True,False,False,False,False,True,True,False,...,False,False,False,False,False,True,False,False,True,4.4
34033,PAAA003039,820535039,True,False,False,True,False,True,True,False,...,False,False,True,False,False,False,False,False,True,4.8
34034,PAAA00305B,820534432,True,False,False,False,True,True,True,False,...,False,False,False,False,False,False,False,False,True,3.6
34035,PAAA012034,820534189,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,2.0
34036,PAAA012056,820534190,True,False,False,False,False,True,True,False,...,False,False,True,False,False,False,False,False,True,4.4
34037,PAAA01303X,820534724,True,False,False,False,False,True,True,False,...,False,False,True,False,False,False,False,False,True,4.4
34038,PAAA01303X,820534723,False,False,False,True,False,True,True,False,...,False,False,True,False,False,False,False,False,True,4.4
34039,PAAA013041,820533328,True,False,False,False,True,True,True,True,...,False,False,True,False,False,False,False,False,True,5.6
34040,PAAA017037,820534531,True,False,False,False,False,True,True,True,...,False,False,True,False,False,False,False,False,True,5.2
34041,PAAA017048,820533387,True,False,False,True,True,True,False,True,...,False,False,False,False,False,False,False,False,True,3.6


Il file <b>Output_rank</b> contiene:
- Codice della scuola,
- Codice edificio,
- Infrastrutture presenti, come ad esempio aula magna, mensa, spazi amministrativi ecc.
- Rank (valutazione)
